### Imports

In [28]:
import requests
from dotenv import load_dotenv
import os
from geopy.geocoders import Nominatim
import psycopg2

load_dotenv()
DB_PASSWORD = os.getenv("DB_PASSWORD")

### Defining custom header for MET Weather API.
#### The MET Weather API is an interface to a selection of data produced by MET Norway.

In [30]:
headers = {
    "User-Agent": "MyWeatherApp/1.0 puskarpy"
}

#### Get latitude and Longitude

In [31]:
def GetLatAndLon(place):
    geolocator = Nominatim(user_agent="weather-project")
    location = geolocator.geocode(place)
    if location:
        print(location.address)
        return location.latitude, location.longitude
    else:
        return None, None

#### Get all weather info 

In [32]:
def GetWeather(place):
    lat, lon = GetLatAndLon(place)
    if lat == None:
        return None
        
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        return res.json()
    else:
        print(f"Error {res.ststus_code}")
        print(res.text)
        return None

#### Get weather infos.

In [33]:
def GetWeatherInfo(data):
    try:
        timeseries = data['properties']['timeseries'][0]
        timestamp = timeseries['time']
        details = timeseries['data']['instant']['details']
        return {
            "temperature": details.get("air_temperature"),
            "humidity": details.get("relative_humidity"),
            "wind_speed": details.get("wind_speed"),
            "pressure": details.get("air_pressure_at_sea_level"),
            "timestamp": timestamp
            }
    except KeyError:
        return f"{param} not found in data"

In [15]:
# cursor.execute("""CREATE TABLE weather_data(
# id SERIAL PRIMARY KEY,
# city VARCHAR(100),
# latitude FLOAT,
# longitude FLOAT,
# temperature FLOAT,
# humidity FLOAT,
# wind_speed FLOAT,
# pressure FLOAT,
# created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
# data_retrieved_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
# )""")

#### Insert data into database.

In [36]:
def InsertWeatherData(city, lat, lon, weather_data):
    conn = psycopg2.connect(database="postgres", host="localhost", user="postgres", password=DB_PASSWORD, port="5432")
    cursor = conn.cursor()
    cursor.execute("""
    INSERT INTO weather_data
    (city, latitude, longitude, temperature, humidity, wind_speed, pressure, created_at)
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        city,
        lat,
        lon,
        weather_data["temperature"],
        weather_data["humidity"],
        weather_data["wind_speed"],
        weather_data["pressure"],
        weather_data["timestamp"],
    ))
    
    conn.commit()
    conn.close()
    cursor.close()
    print(f"Data inserted for {city}.")

In [37]:
city = "Kathmandu"
data = GetWeather(city)
lat, lon = GetLatAndLon(city)
if data:
    weather_values = GetWeatherInfo(data)
    InsertWeatherData(city, lat, lon, weather_values)

काठमाडौँ महानगरपालिका, काठमाडौं जिल्ला, बागमती प्रदेश, नेपाल
काठमाडौँ महानगरपालिका, काठमाडौं जिल्ला, बागमती प्रदेश, नेपाल
Data inserted for Kathmandu.
